In [8]:
!pip install yt-dlp
!pip install whisperx
!pip install streamlit
!pip install pydub
!pip install pyngrok
!pip install pytube


In [ ]:
nltk.data.path.append("C:\\Users\\angad\\Downloads\\nltk_data")
nltk.download("punkt", download_dir="C:\\Users\\angad\\Downloads\\nltk_data", force=True)
nltk.download("stopwords", download_dir="C:\\Users\\angad\\Downloads\\nltk_data", force=True)

In [9]:
import yt_dlp
import os

def download_youtube_audio(youtube_url, output_file='audio'):
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': output_file,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }]
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([youtube_url])

    return output_file


In [10]:
import whisperx

def transcribe_audio(file_path):
    model = whisperx.load_model("large")
    audio = whisperx.load_audio("/content/audio.mp3")

    result = model.transcribe(audio, language='en')

    return result["text"]


In [11]:
%%writefile app.py

import streamlit as st
import yt_dlp
import whisperx
import torch
import os
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from collections import Counter

def download_youtube_audio(youtube_url, output_file='/content/audio.mp3'):
    output_file = os.path.splitext(output_file)[0] + '.mp3'

    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': output_file,  
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }]
    }
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([youtube_url])
        print(f"Downloaded audio to {output_file}")
    except Exception as e:
        print(f"Error downloading audio: {e}")
        return None  
    if not os.path.exists(output_file):
        print(f"Audio file not found at {output_file}.")
        return None

    return output_file

def transcribe_audio(file_path):
    device = "cuda" if torch.cuda.is_available() else "cpu" 
    model = whisperx.load_model("large", device=device)
    audio = whisperx.load_audio("/content/audio.mp3.mp3")
    result = model.transcribe(audio)
    print("Transcription result:", result)  
    if 'text' in result:
        return result['text']
    elif 'segments' in result and len(result['segments']) > 0:
        # If 'segments' exist, join their text
        transcription = ' '.join(segment['text'] for segment in result['segments'])
        return transcription
    else:
        raise KeyError("The key 'text' was not found in the transcription result.")

def generate_summary(transcription, num_sentences=5):
    try:
        sentences = sent_tokenize(transcription)
        stop_words = set(stopwords.words("english"))
        words = word_tokenize(transcription.lower())
        filtered_words = [word for word in words if word.isalnum() and word not in stop_words]
        word_frequencies = Counter(filtered_words)
        sentence_scores = {}
        for sentence in sentences:
            for word in word_tokenize(sentence.lower()):
                if word in word_frequencies:
                    sentence_scores[sentence] = sentence_scores.get(sentence, 0) + word_frequencies[word]
        ranked_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)
        summary = " ".join(ranked_sentences[:num_sentences])
        return summary
    except Exception as e:
        return f"Error during summarization: {e}"

def main():
st.title("YouTube Video Transcription and Summarization")
youtube_link = st.text_input("Enter YouTube Video Link:")
    if st.button("Transcribe"):
        with st.spinner("Downloading audio..."):
            audio_file = download_youtube_audio(youtube_url)

        with st.spinner("Transcribing audio..."):
            transcription = transcribe_audio(audio_file)

        st.write("### Transcription:")
        st.text_area("Transcribed Text", transcription, height=400)
        st.download_button('Download Transcription', transcription)
if youtube_link:
    video_id = youtube_link.split("=")[1]
    st.image(f"http://img.youtube.com/vi/{video_id}/0.jpg", use_container_width=True)

if st.button("Get Detailed Notes"):
    if st.button("Transcribe"):
        with st.spinner("Downloading audio..."):
            audio_file = download_youtube_audio(youtube_url)

        with st.spinner("Transcribing audio..."):
            transcription = transcribe_audio(audio_file)
    if transcription:
        st.markdown("## Full Transcription:")
        st.write(transcription)
        st.markdown("## Summary:")
        summary = generate_summary(transcription)
        st.write(summary)
    else:
        st.error("Could not retrieve transcript. Please check the video link.")

if __name__ == '__main__':
    main()


Overwriting app.py


In [12]:
from pyngrok import ngrok
ngrok.set_auth_token("2nCgSm4c6FXtWmXG30VmrOCwqnw_wi3nX4u5RFUESEEbrvMU")
public_url = ngrok.connect(8501)
print(f"Public URL: {public_url}")

Public URL: NgrokTunnel: "https://931a-34-173-134-139.ngrok-free.app" -> "http://localhost:8501"


In [13]:
!streamlit run app.py &





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.173.134.139:8501

2024-10-10 05:11:10.971384: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-10 05:11:11.003711: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-10 05:11:11.013016: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-10 05:11:12.706264: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/pyannote/audio/core/io.p